# Set up

In [ ]:
import epics
import torch
torch.set_default_dtype(torch.float64)
# torch.set_default_tensor_type('torch.DoubleTensor')
run_dir = '/home/physics/ml_tuning/20250611_LCLS_Injector/'

# Create screen

In [ ]:
from lcls_tools.common.devices.reader import create_screen
screen = create_screen(area="DL1", name="OTR2")

# Take background images

In [ ]:
## measure background
shutter_pv = "IOC:BSY0:MP01:MSHUTCTL"

import numpy as np
from time import sleep
epics.caput(shutter_pv,0) 
sleep(1)

background_images = []
for i in range(20):
    background_images += [screen.image]
    sleep(0.2)

background_image = np.mean(background_images, axis=0)

epics.caput(shutter_pv,1) 
sleep(1)

plt.imshow(background_image)

# Define image_processor and beamsize_measurement

In [ ]:
from ml_tto.automatic_emittance.screen_profile import ScreenBeamProfileMeasurement
from lcls_tools.common.image.roi import ROI
from lcls_tools.common.image.processing import ImageProcessor
from ml_tto.automatic_emittance.image_projection_fit import RecursiveImageProjectionFit

image_processor = ImageProcessor(
    background_image=background_image,
    roi=ROI(center=[600,400], extent=[600, 600]),
)

image_projection_fit = RecursiveImageProjectionFit()

beamsize_measurement = ScreenBeamProfileMeasurement(
    device=screen,
    image_processor = image_processor,
    beam_fit=image_projection_fit
)
beamsize_measurement.measure()

# Plot raw screen image

In [ ]:
plt.imshow(screen.image)

# Test image fitting

In [ ]:
from ml_tto.automatic_emittance.plotting import plot_image_projection_fit
result = image_projection_fit.fit_image(image_processor.auto_process(screen.image))
plot_image_projection_fit(result)

# Define evaluate function

In [ ]:
def evaluate(inputs: dict) -> dict:
    result = beamsize_measurement.measure()
    # process results
    xrms = result.rms_sizes[:, 0] * beamsize_measurement.device.resolution * 1e-3 # beam size in millimeters
    yrms = result.rms_sizes[:, 1] * beamsize_measurement.device.resolution * 1e-3 # beam size in millimeters
    xrms_sq = xrms**2
    yrms_sq = yrms**2
    return {'xrms_sq': xrms_sq, 
            'yrms_sq': yrms_sq,
           }

# List variables

In [ ]:
var_names = ['SOLN:IN20:121:BCTRL',
 'QUAD:IN20:121:BCTRL',
 'QUAD:IN20:122:BCTRL',
 'QUAD:IN20:361:BCTRL',
 'QUAD:IN20:371:BCTRL',
 'QUAD:IN20:425:BCTRL',
 'QUAD:IN20:441:BCTRL',
 'QUAD:IN20:511:BCTRL',
 'QUAD:IN20:525:BCTRL'
]
meas_quad = 'QUAD:IN20:525:BCTRL'
init_values = dict(zip(var_names, epics.caget_many(var_names)))
print(init_values)

# grab initial variable settings

In [ ]:
import pickle

with open('init_values.pickle', 'rb') as handle:
    init_values = pickle.load(handle)

In [ ]:
proposed_variable_ranges = {}

for key, value in init_values.items():

	proposed_variable_ranges[key] = sorted([value * 0.9, value * 1.1])

print(proposed_variable_ranges)

In [ ]:
variables = 

# Define VOCS

In [ ]:
# construct vocs
vocs = VOCS(
    variables = variables,
    observables = ['xrms_sq', 'yrms_sq'],
    constraints = 
)

print('variable_names =', vocs.variable_names)
print('meas_quad =', "'" + meas_param + "'")
print('domain =\n', vocs.bounds.T)

# Identify measurement quad dimension in Xopt model

In [ ]:
meas_dim = vocs.variable_names.index(meas_quad)

# define (quadratic x matern) product kernel model constructor

In [ ]:
from gpytorch.kernels import MaternKernel, PolynomialKernel, ScaleKernel
from gpytorch.priors.torch_priors import GammaPrior

from xopt.generators.bayesian.models.standard import StandardModelConstructor

# prepare custom covariance module
tuning_dims = list(range(vocs.n_variables))
tuning_dims.remove(meas_dim)
covar_module_x = (MaternKernel(ard_num_dims=len(tuning_dims), 
                              active_dims=tuning_dims, 
                              lengthscale_prior=None) * 
                              PolynomialKernel(power=2, active_dims=[meas_dim])
                 )

scaled_covar_module_x = ScaleKernel(covar_module_x)#, outputscale_prior=GammaPrior(2.0, 0.15))
covar_module_y = (MaternKernel(ard_num_dims=len(tuning_dims), 
                              active_dims=tuning_dims, 
                              lengthscale_prior=None) * 
                              PolynomialKernel(power=2, active_dims=[meas_dim])
                 )
scaled_covar_module_y =  ScaleKernel(covar_module_y)#, outputscale_prior=GammaPrior(2.0, 0.15))

covar_module_dict = {'xrms_sq': scaled_covar_module_x,
                     'yrms_sq': scaled_covar_module_y}

model_constructor = StandardModelConstructor(covar_modules=covar_module_dict)

# Define numerical optimizer (for Xopt acquisition function optimization)

In [ ]:
from xopt.numerical_optimizer import LBFGSOptimizer
numerical_optimizer = LBFGSOptimizer(n_restarts=10,
                                    max_time=2)

# OTR2 emittance measurement config values (measurement quad : QE04 / QUAD:IN20:525)

In [ ]:
energy = 0.135e9 # eV
q_len = 0.108 # measurement quad effective length in meters

design_twiss = {}
design_twiss['beta_x'] = 1.113
design_twiss['alpha_x'] = -.069
design_twiss['beta_y'] = 1.113
design_twiss['alpha_y'] = -.07

# 2x2 rmat for x (from END of measurement quad to screen)
rmat_x = np.array([[1., 2.26],
                   [0, 1.]]) 
# 2x2 rmat for y (from END of measurement quad to screen)
rmat_y = np.array([[1., 2.26],
                   [0, 1.]]) 

# Define emittance algorithm and construct BaxGenerator

In [ ]:
from bax_algorithms.emittance import PathwiseMinimizeEmittance
from bax_algorithms.pathwise.optimize import DifferentialEvolution
from xopt.generators.bayesian.bax_generator import BaxGenerator
from xopt.evaluator import Evaluator
from xopt import Xopt

#Prepare Algorithm
algo_kwargs = {
        'x_key': 'xrms_sq',
        'y_key': 'yrms_sq',
        'energy': energy,
        'q_len': q_len,
        'rmat_x': rmat_x,
        'rmat_y': rmat_y,
        'twiss0_x': torch.tensor([design_twiss['beta_x'], design_twiss['alpha_x']]),
        'twiss0_y': torch.tensor([design_twiss['beta_y'], design_twiss['alpha_y']]),
        'n_samples': 3,
        'meas_dim': meas_dim,
        'n_steps_measurement_param': 11,
        'use_bmag': True,
        'observable_names_ordered': ['xrms_sq','yrms_sq'],
        'optimizer': DifferentialEvolution(minimize=True, maxiter=10, verbose=True),
        # 'maxiter_fit': 10,
}
algo = PathwiseMinimizeEmittance(**algo_kwargs)

#construct BAX generator
generator = BaxGenerator(vocs=vocs, 
                         gp_constructor=model_constructor, 
                         numerical_optimizer=numerical_optimizer,
                         algorithm=algo,
                         n_interpolate_points=5,
                         use_cuda=False)

generator.gp_constructor.use_low_noise_prior = False

#construct evaluator
evaluator = Evaluator(function=evaluate)

#construct Xopt optimizer
X = Xopt(evaluator=evaluator, generator=generator, vocs=vocs)

# Evaluate initial data

# visualize initial model

In [ ]:
reference_points = init_values

In [ ]:
from xopt.generators.bayesian.visualize import visualize_generator_model
X.generator.train_model()
visualize_generator_model(X.generator, 
                          variable_names=['QUAD:IN20:525:BCTRL'], 
                            reference_point=reference_point,
                          show_acquisition=False)
visualize_generator_model(X.generator, 
                          variable_names=['SOLN:IN20:121:BCTRL','QUAD:IN20:525:BCTRL'], 
                            reference_point=reference_point,
                          show_acquisition=False)

In [ ]:
from bax_algorithms.visualize import visualize_virtual_measurement_result

fig, ax = visualize_virtual_measurement_result(X.generator, 
                            variable_names=['SOLN:IN20:121:BCTRL'],
                            reference_point=reference_point,
                            n_grid=100,
                            n_samples=100,
                            result_keys=['objective','emit_x','emit_y','bmag_x','bmag_y'],
                                     )

# Run BAX steps

In [ ]:
for i in range(5):
    print(i)
    start = time.time()
    X.step()
    print(time.time() - start)

# Visualize model

In [ ]:
X.generator.train_model()
visualize_generator_model(X.generator, 
                          variable_names=['QUAD:IN20:525:BCTRL'], 
                            reference_point=reference_point,
                          show_acquisition=False)
visualize_generator_model(X.generator, 
                          variable_names=['SOLN:IN20:121:BCTRL','QUAD:IN20:525:BCTRL'], 
                            reference_point=reference_point,
                          show_acquisition=False)

In [ ]:
fig, ax = visualize_virtual_measurement_result(X.generator, 
                            variable_names=['SOLN:IN20:121:BCTRL'],
                            reference_point=reference_point,
                            n_grid=100,
                            n_samples=100,
                            result_keys=['objective','emit_x','emit_y','bmag_x','bmag_y'],
                                     )

# Get best BAX estimate (run algorithm on model posterior mean)

In [ ]:
from bax_algorithms.utils import get_bax_mean_prediction, tuning_input_tensor_to_dict
mean_optimizer = DifferentialEvolution(minimize=True, popsize=100, maxiter=100, verbose=True)
x_tuning = get_bax_mean_prediction(X.generator, mean_optimizer)
x_tuning_dict = tuning_input_tensor_to_dict(X.generator, x_tuning)
print(x_tuning)
print(x_tuning_dict)